In [1]:
import pandas as pd
from datetime import datetime
import time
from itertools import islice

In [2]:
def append_new_row(data_frame, row):
    data_frame = data_frame.append({'lat_begin' : row['lat'], 'lng_begin' : row['lng'],
                                'lat_end' : 0, 'lng_end' : 0,
                                'ride_begin' : row['parking_end'], 'ride_end' : 0,
                                'ride_time' : 0,
                                'uid_begin' : row['uid'], 'uid_end' : 0, 'standing_before_ride' : row['parked_time'],
                                'name' : row['name'], 'address' : row['address'],
                                'bike' : row['bike'], 'bike_racks' : row['bike_racks'], 
                                'bikes' : row['bikes'], 'booked_bikes' : row['booked_bikes'], 
                                'free_racks' : row['free_racks'], 'free_special_racks' : row['free_special_racks'],
                                'maintenance' : row['maintenance'], 'number' : row['number'], 
                                'place_type' : row['place_type'], 'rack_locks' : row['rack_locks'],
                                'special_racks' : row['special_racks'], 'spot' : row['spot'], 
                                'terminal_type' : row['terminal_type'], 'city' : row['city'],
                                'country_code' : row['country_code'], 'company' : row['company'],
                                'timezone' : row['timezone'],
                                'available_bikes' : row['available_bikes']}, ignore_index=True)
    return data_frame

In [3]:
def parse_timestamp(el):
    a = el.split('.')[0]
    b = datetime.strptime(a, fmt)
    return b

In [4]:
fmt = '%Y-%m-%d %H:%M:%S'

### Processes the data in order to create new column  'ride_time_minutes' 

#### As URL set one of the following:
1. 'cleaned_data_nextbike_yy-mm.csv.gz' if you want to work with original cleaned data 
2. 'processed_data_nextbike_yy-mm.csv.gz' if you want to work with data that has GPS errors removed 

CHANGE NAME OF OUTPUT TO DESCRIBE THE CHOICE

In [5]:
url = "/bigdata/jelicicna/output_datasets/processed_data_nextbike_2020-09.csv.gz"
df = pd.read_csv(url, sep=';', low_memory=False)

In [6]:
df['parking_begin'] = df['parking_begin'].map(parse_timestamp)
df['parking_end'] = df['parking_end'].map(parse_timestamp)

In [7]:
pd.set_option('display.max_columns', None)

In [8]:
df["uid"].duplicated().any() #checking if cleaning in notebook @3 was done - there should be no duplicates

False

In [9]:
df.head()

,lat,lng,parking_begin,parking_end,parked_time,uid,name,address,available_bikes,maintenance,bike,bike_racks,bikes,booked_bikes,free_racks,free_special_racks,number,place_type,rack_locks,special_racks,spot,terminal_type,city,country_code,company,timezone
0,50.958746,6.994779,2020-09-01 01:45:02,2020-09-01 06:20:03,275.0,35281594,BIKE 22754,NaN,1197,False,True,0,1,0,0,0,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin
1,50.961834,7.003976,2020-09-01 08:59:03,2020-09-01 16:25:01,446.0,35286871,BIKE 22754,NaN,1177,False,True,0,1,0,0,0,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin
2,50.958002,6.995119,2020-09-01 22:50:02,2020-09-01 22:57:02,7.0,35310037,BIKE 22754,NaN,1194,False,True,0,1,0,0,0,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin
3,50.957624,6.994782,2020-09-01 22:58:02,2020-09-02 06:16:02,438.0,35328407,BIKE 22754,NaN,1192,False,True,0,1,0,0,0,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin
4,50.968676,7.013570,2020-09-02 07:46:02,2020-09-02 16:46:01,540.0,35336700,BIKE 22754,NaN,1205,False,True,0,1,0,0,0,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin


In [10]:
new_df = pd.DataFrame(columns= ['lat_begin', 'lng_begin', 'lat_end', 'lng_end', 'ride_begin', 'ride_end', 'ride_time', 
                                'uid_begin', 'uid_end', 'standing_before_ride', 'address', 'name', 'available_bikes', 'bike', 'bike_racks', 'bikes', 
                                'booked_bikes', 'free_racks', 'free_special_racks', 'maintenance', 'number', 
                                'place_type', 'rack_locks', 'special_racks', 'spot', 'terminal_type', 'city',
                                'country_code', 'company', 'timezone'] ) #empty dataframe with the new column structure

In [11]:
bikesToList = df['name'].tolist()
bikesList = list(set(bikesToList))

In [12]:
for name in bikesList: #repeat for each bike/bike name
    single_bike_records = df[df['name']==name]
    first_row_data = single_bike_records.iloc[0]
    last_end_time = first_row_data['parking_end']

    new_df = append_new_row(new_df, first_row_data)

    for index, row in islice(single_bike_records.iterrows(), 1, None):

        d1_ts = time.mktime(df['parking_begin'][index].timetuple())
        d2_ts = time.mktime(last_end_time.timetuple())

        minsDiff = round((int(d1_ts-d2_ts) / 60), 0)

        new_df.iloc[-1, new_df.columns.get_loc('ride_time')] = minsDiff
        new_df.iloc[-1, new_df.columns.get_loc('lat_end')] = row['lat']
        new_df.iloc[-1, new_df.columns.get_loc('lng_end')] = row['lng']
        new_df.iloc[-1, new_df.columns.get_loc('ride_end')] = row['parking_begin']
        new_df.iloc[-1, new_df.columns.get_loc('uid_end')] = row['uid']

        last_end_time = row['parking_end']
        
        new_df = append_new_row(new_df, row)

In [13]:
new_df.head()

,lat_begin,lng_begin,lat_end,lng_end,ride_begin,ride_end,ride_time,uid_begin,uid_end,standing_before_ride,address,name,available_bikes,bike,bike_racks,bikes,booked_bikes,free_racks,free_special_racks,maintenance,number,place_type,rack_locks,special_racks,spot,terminal_type,city,country_code,company,timezone
0,50.915359,6.928819,50.9343,6.93828,2020-09-01 00:47:02,2020-09-01 01:02:02,15.0,35268434,35283100,47.0,NaN,BIKE 21014,1177,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin
1,50.934330,6.938283,50.9488,6.99105,2020-09-01 01:52:02,2020-09-01 02:19:02,27.0,35283100,35284237,50.0,NaN,BIKE 21014,1189,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin
2,50.948824,6.991045,50.9533,7.01382,2020-09-01 21:37:03,2020-09-03 03:22:02,1785.0,35284237,35395707,1158.0,NaN,BIKE 21014,1198,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin
3,50.953270,7.013817,50.9378,6.98499,2020-09-04 19:48:02,2020-09-04 20:12:03,24.0,35395707,35505514,2426.0,NaN,BIKE 21014,1235,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin
4,50.937835,6.984989,50.937,6.97449,2020-09-04 21:13:02,2020-09-04 21:21:02,8.0,35505514,35510956,61.0,NaN,BIKE 21014,1210,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin


In [14]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64454 entries, 0 to 64453
Data columns (total 30 columns):
lat_begin               64454 non-null float64
lng_begin               64454 non-null float64
lat_end                 64454 non-null object
lng_end                 64454 non-null object
ride_begin              64454 non-null datetime64[ns]
ride_end                64454 non-null object
ride_time               64454 non-null float64
uid_begin               64454 non-null object
uid_end                 64454 non-null int64
standing_before_ride    64454 non-null float64
address                 473 non-null object
name                    64454 non-null object
available_bikes         64454 non-null object
bike                    64454 non-null object
bike_racks              64454 non-null object
bikes                   64454 non-null object
booked_bikes            64454 non-null object
free_racks              64454 non-null object
free_special_racks      64454 non-null object
maintena

###  Prevous code assignes 0 values to some features that should be updated trough itteration to contain acctual end values of one ride.
### As a result for each bike there will be one extra row that has not updated values, meaning itteration has reached the end for selected bike and 0 values were not updated. 
### In the following cells we check for rows like that - idealy number of rows shoud match number of bikes

In [15]:
control_df = new_df[new_df['ride_end'] == 0]
control_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 994 entries, 59 to 64453
Data columns (total 30 columns):
lat_begin               994 non-null float64
lng_begin               994 non-null float64
lat_end                 994 non-null object
lng_end                 994 non-null object
ride_begin              994 non-null datetime64[ns]
ride_end                994 non-null object
ride_time               994 non-null float64
uid_begin               994 non-null object
uid_end                 994 non-null int64
standing_before_ride    994 non-null float64
address                 44 non-null object
name                    994 non-null object
available_bikes         994 non-null object
bike                    994 non-null object
bike_racks              994 non-null object
bikes                   994 non-null object
booked_bikes            994 non-null object
free_racks              994 non-null object
free_special_racks      994 non-null object
maintenance             994 non-null object
nu

In [16]:
len(bikesList)

994

In [17]:
incorrect_indexes = list(control_df.index.values)
len(incorrect_indexes)

994

In [18]:
for i in incorrect_indexes:
    if i in new_df.index.values:
        new_df.drop(i, inplace=True)

In [19]:
new_df = new_df.reset_index()

In [20]:
new_df = new_df.drop(['index'], axis=1)

In [21]:
#new_df['ride_time'].value_counts(dropna=False)

In [22]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63460 entries, 0 to 63459
Data columns (total 30 columns):
lat_begin               63460 non-null float64
lng_begin               63460 non-null float64
lat_end                 63460 non-null object
lng_end                 63460 non-null object
ride_begin              63460 non-null datetime64[ns]
ride_end                63460 non-null object
ride_time               63460 non-null float64
uid_begin               63460 non-null object
uid_end                 63460 non-null int64
standing_before_ride    63460 non-null float64
address                 429 non-null object
name                    63460 non-null object
available_bikes         63460 non-null object
bike                    63460 non-null object
bike_racks              63460 non-null object
bikes                   63460 non-null object
booked_bikes            63460 non-null object
free_racks              63460 non-null object
free_special_racks      63460 non-null object
maintena

In [23]:
new_df.to_csv('/bigdata/jelicicna/output_datasets/rides_table_nextbike_2020-09.csv', sep=';', index=False) 